In [1]:
import numpy as np
import cv2
from project.common.camera.distance import judge_distance
from project.generated.project.common.proto.Inference_pb2 import InferenceList

# Load images
left_img = cv2.imread("./output/left.jpg")
right_img = cv2.imread("./output/right.jpg")

if left_img is None or right_img is None:
    raise FileNotFoundError("Left or right image could not be loaded. Check the file paths.")

# Store original dimensions before resizing
original_height, original_width = left_img.shape[:2]

# Resize images to a fixed size for processing
left_img = cv2.resize(left_img, (640, 480))
right_img = cv2.resize(right_img, (640, 480))

# Parse inference data
out: None | InferenceList = None
with open('output/left_pos.txt', 'rb') as file:
    content = file.read()
    out = InferenceList.FromString(content)

if out is None or not out.inferences:
    raise ValueError("InferenceList is empty or invalid.")

# Extract bounding box
bbox = []
for i in out.inferences:
    for j in i.bounding_box:
        bbox.append(j)

if len(bbox) < 4:
    raise ValueError("Bounding box data is incomplete.")

# Scale bounding box coordinates to match resized image dimensions
bbox = [int(coord * (640 / original_width)) if idx % 2 == 0 else int(coord * (480 / original_height)) for idx, coord in enumerate(bbox)]

# Ensure bounding box dimensions are valid
if not (0 <= bbox[0] < bbox[2] <= 640 and 0 <= bbox[1] < bbox[3] <= 480):
    raise ValueError("Bounding box is out of bounds or invalid after resizing.")

# Parameters
baseline = 0.023
focal_length_left = 1457.5931734170763
focal_length_right = 1487.1124346674526
focal_length = (focal_length_left + focal_length_right) / 2

camera_matrix_left = np.array([
    [1457.5931734170763, 0.0, 997.1660947238562],
    [0.0, 1401.1036215895347, 539.0780233970917],
    [0.0, 0.0, 1.0],
])
distortion_left = np.array([
    -0.3699966980628528,
    0.1821265728482258,
    -0.000017146753545554133,
    0.001312177635207973,
    0.060869889777672416,
])

camera_matrix_right = np.array([
    [1487.1124346674526, 0.0, 945.122412363984],
    [0.0, 1434.705975660968, 547.8805261625706],
    [0.0, 0.0, 1.0],
])
distortion_right = np.array([
    -0.4513475113205368,
    0.21139658631382788,
    -0.0028846973373456855,
    0.0021349747481580624,
    -0.055584296827295585,
])

rotation_matrix = np.eye(3)
translation_vector = np.array([[baseline], [0], [0]])

# Compute distance
try:
    distance = judge_distance(
        left_img, right_img, (bbox[0], bbox[1], bbox[2], bbox[3]), focal_length, baseline,
        camera_matrix_left, camera_matrix_right,
        distortion_left, distortion_right,
        rotation_matrix, translation_vector
    )
    if distance:
        print(f"Distance to person: {distance:.2f} meters")
    else:
        print("Unable to compute distance (invalid disparity).")
except Exception as e:
    print(f"Error during distance computation: {e}")

Unable to compute distance (invalid disparity).
